In [1]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import pandas as pd
import json

In [26]:
openai_api_key = '...'

In [3]:
# Temp = 0 so that we get clean information without a lot of creativity
chat_model = ChatOpenAI(temperature=0, openai_api_key=openai_api_key, max_tokens=1000)

D:\Python\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:
# How you would like your response structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="input_industry", description="This is the input_industry from the user"),
    ResponseSchema(name="standardized_industry", description="This is the industry you feel is most closely matched to the users input"),
    ResponseSchema(name="match_score",  description="A score 0-100 of how close you think the match is between user input and your match")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [5]:
# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (output_parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"input_industry": string  // This is the input_industry from the user
	"standardized_industry": string  // This is the industry you feel is most closely matched to the users input
	"match_score": string  // A score 0-100 of how close you think the match is between user input and your match
}
```


In [6]:
template = """
You will be given a series of industry names from a user.
Find the best corresponding match on the list of standardized names.
The closest match will be the one with the closest semantic meaning. Not just string similarity.

{format_instructions}

Wrap your final output with closed and open brackets (a list of json objects)

input_industry INPUT:
{user_industries}

STANDARDIZED INDUSTRIES:
{standardized_industries}

YOUR RESPONSE:
"""

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)  
    ],
    input_variables=["user_industries", "standardized_industries"],
    partial_variables={"format_instructions": format_instructions}
)

In [15]:
# Get your standardized names. You can swap this out with whatever list you want!
df = pd.read_csv('LinkedInIndustries.csv')
standardized_industries = ", ".join(df['Industry'].values)
standardized_industries

'Corporate Services, Recreation & Travel, Legal, Wellness & Fitness, Entertainment, Consumer Goods, Design, Arts, Manufacturing, Finance, Health Care, Construction, Nonprofit, Real Estate, Software & IT Services, Hardware & Networking, Agriculture, Education, Public Administration, Transportation & Logistics, Public Safety, Media & Communications, Energy & Mining, Retail'

In [16]:
# Your user input

user_input = "air LineZ, airline, aviation, planes that fly, farming, bread, wifi networks, twitter media agency"

_input = prompt.format_prompt(user_industries=user_input, standardized_industries=standardized_industries)


print (f"There are {len(_input.messages)} message(s)")
print (f"Type: {type(_input.messages[0])}")
print ("---------------------------")
print (_input.messages[0].content)

There are 1 message(s)
Type: <class 'langchain_core.messages.human.HumanMessage'>
---------------------------

You will be given a series of industry names from a user.
Find the best corresponding match on the list of standardized names.
The closest match will be the one with the closest semantic meaning. Not just string similarity.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"input_industry": string  // This is the input_industry from the user
	"standardized_industry": string  // This is the industry you feel is most closely matched to the users input
	"match_score": string  // A score 0-100 of how close you think the match is between user input and your match
}
```

Wrap your final output with closed and open brackets (a list of json objects)

input_industry INPUT:
air LineZ, airline, aviation, planes that fly, farming, bread, wifi networks, twitter media agency

STANDARDIZED INDU

In [17]:
output = chat_model(_input.to_messages())

D:\Python\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [18]:
print (type(output))
print (output.content)

<class 'langchain_core.messages.ai.AIMessage'>
```json
[
	{
		"input_industry": "air LineZ",
		"standardized_industry": "Transportation & Logistics",
		"match_score": "80"
	},
	{
		"input_industry": "airline",
		"standardized_industry": "Transportation & Logistics",
		"match_score": "90"
	},
	{
		"input_industry": "aviation",
		"standardized_industry": "Transportation & Logistics",
		"match_score": "95"
	},
	{
		"input_industry": "planes that fly",
		"standardized_industry": "Transportation & Logistics",
		"match_score": "85"
	},
	{
		"input_industry": "farming",
		"standardized_industry": "Agriculture",
		"match_score": "90"
	},
	{
		"input_industry": "bread",
		"standardized_industry": "Consumer Goods",
		"match_score": "80"
	},
	{
		"input_industry": "wifi networks",
		"standardized_industry": "Hardware & Networking",
		"match_score": "85"
	},
	{
		"input_industry": "twitter media agency",
		"standardized_industry": "Media & Communications",
		"match_score": "95"
	}
]
```


In [19]:
if "```json" in output.content:
    json_string = output.content.split("```json")[1].strip()
else:
    json_string = output.content

In [20]:
print(output.content)

```json
[
	{
		"input_industry": "air LineZ",
		"standardized_industry": "Transportation & Logistics",
		"match_score": "80"
	},
	{
		"input_industry": "airline",
		"standardized_industry": "Transportation & Logistics",
		"match_score": "90"
	},
	{
		"input_industry": "aviation",
		"standardized_industry": "Transportation & Logistics",
		"match_score": "95"
	},
	{
		"input_industry": "planes that fly",
		"standardized_industry": "Transportation & Logistics",
		"match_score": "85"
	},
	{
		"input_industry": "farming",
		"standardized_industry": "Agriculture",
		"match_score": "90"
	},
	{
		"input_industry": "bread",
		"standardized_industry": "Consumer Goods",
		"match_score": "80"
	},
	{
		"input_industry": "wifi networks",
		"standardized_industry": "Hardware & Networking",
		"match_score": "85"
	},
	{
		"input_industry": "twitter media agency",
		"standardized_industry": "Media & Communications",
		"match_score": "95"
	}
]
```
